# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
#from perceptronac.perfect_AC import perfect_AC
import torch
import os

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [27]:
#X is context and y is bitstream to encode
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

X = (X > 0).astype(int)

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)).astype(int) # signal 
yt = yy[0:L] > 0 # train on the first part 
yc = yy[L:L+L] > 0 # encode the second part
Xt = X[0:L,0:N] # truncated X for training 
Xc = X[L:L+L,0:N] # truncated X for coding

In [28]:
#Xc

In [ ]:
#X is context and y is bitstream to encode
L = 100000 # how many samples 
#N = 10 # order of the AR
first = True
for pc_num in range(11, 15):
    if first:
        my_data = genfromtxt(f'../SPIHT_dataset/ricardo/SPIHT_bits_with_context_ricardo_v9_Y_{pc_num}_v10.csv', delimiter=',')
        first = False
    else:
        my_data = np.append(my_data,
                            genfromtxt(f'../SPIHT_dataset/ricardo/SPIHT_bits_with_context_ricardo_v9_Y_{pc_num}_v10.csv', delimiter=','),
                            axis=0)
    
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:7].astype(float)
extra_context = my_data[:, 7:].astype(float)
#extra_context = extra_context.reshape((len(extra_context), 1))
bitstream = bitstream.reshape((len(bitstream), 1))


my_data_test = genfromtxt('../SPIHT_dataset/andrew/SPIHT_bits_with_context_andrew_Y_10_v10.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:7].astype(float)
extra_context_test = my_data_test[:, 7:].astype(float)
#extra_context_test = extra_context_test.reshape((len(extra_context_test), 1))
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))

In [ ]:
yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context/abs(extra_context).max(axis=0), axis=1)
#Xt = extra_context/abs(extra_context).max(axis=0)
Xt[np.isnan(Xt)] = 0
Xc = np.append(Xc, extra_context_test/abs(extra_context_test).max(axis=0), axis=1)
#Xc = extra_context_test/abs(extra_context_test).max(axis=0)
Xc[np.isnan(Xc)] = 0


N = Xc[0, :].size # order of the AR

In [ ]:
N

## Entropia dos dados

In [5]:
yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
#Xt = np.append(Xt, extra_context/abs(extra_context).max(axis=0), axis=1)
Xt[np.isnan(Xt)] = 0
#Xc = np.append(Xc, extra_context_test/abs(extra_context_test).max(axis=0), axis=1)
Xc[np.isnan(Xc)] = 0


N = Xc[0, :].size # order of the AR

In [6]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

NameError: name 'perfect_AC' is not defined

In [7]:
# teste
perfect_AC(yc[:],context_coding(Xc[:, :],context_training(Xc[:, :],yc[:])))

NameError: name 'perfect_AC' is not defined

# Criando classes

In [5]:
import torch

In [6]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [7]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

In [9]:
class MLP_N_64N_32N_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 64*N),
            torch.nn.ReLU(),
            torch.nn.Linear(64*N, 32*N),
            torch.nn.ReLU(),
            torch.nn.Linear(32*N, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

In [10]:
class MLP_N_1024_512_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)


class MLP_N_2048_1024_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [14]:
net = Perceptron(N)

In [15]:
net = net.cuda()

In [63]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [64]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [65]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [66]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.003491003723144531
(compare com a entropia do dataset de treino).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [67]:
for param in net.parameters():
    print(param.data)

tensor([[12.0901, 11.7186, 11.9979, 12.1763, 12.1026]], device='cuda:0')
tensor([41.8704], device='cuda:0')


In [68]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [11]:
#net = Perceptron(N)
#net = MLP_N_64N_32N_1(N)
net = MLP_N_1024_512_1(N) #melhor resultado
#net = MLP_N_2048_1024_1(N)

In [12]:
net = net.cuda()

In [13]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [14]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.000001)

In [ ]:

batch_size = 10000

train_loss, valid_loss = [], []

for epoch in range(150):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : train , loss : 0.9717047951376283
epoch : 0 , phase : valid , loss : 0.9509854270695919
epoch : 1 , phase : train , loss : 0.9497114645081053
epoch : 1 , phase : valid , loss : 0.943273384908049
epoch : 2 , phase : train , loss : 0.9432321312440688
epoch : 2 , phase : valid , loss : 0.9402434393096178
epoch : 3 , phase : train , loss : 0.9409189501607299
epoch : 3 , phase : valid , loss : 0.9389055571568122
epoch : 4 , phase : train , loss : 0.9395846382051576
epoch : 4 , phase : valid , loss : 0.9373177774424886
epoch : 5 , phase : train , loss : 0.9386583164653571
epoch : 5 , phase : valid , loss : 0.9366583947306468
epoch : 6 , phase : train , loss : 0.9378952746717534
epoch : 6 , phase : valid , loss : 0.9361031529829074
epoch : 7 , phase : train , loss : 0.9369317013182733
epoch : 7 , phase : valid , loss : 0.9356766088278893
epoch : 8 , phase : train , loss : 0.9360641699760633
epoch : 8 , phase : valid , loss : 0.9359921471283851
epoch : 9 , phase : train , l

epoch : 75 , phase : train , loss : 0.9265743710787937
epoch : 75 , phase : valid , loss : 0.9278184548266027
epoch : 76 , phase : train , loss : 0.9267429603881178
epoch : 76 , phase : valid , loss : 0.9278159252331205
epoch : 77 , phase : train , loss : 0.9264490973570421
epoch : 77 , phase : valid , loss : 0.9288513084896707
epoch : 78 , phase : train , loss : 0.9265811231461477
epoch : 78 , phase : valid , loss : 0.9280870977884699
epoch : 79 , phase : train , loss : 0.9265303310135962
epoch : 79 , phase : valid , loss : 0.9279657570632792


In [ ]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

### Salva os pesos

In [ ]:
file_name = 'pesos_ricardo_vox9_Y_1024-512.txt'

In [ ]:
torch.save(net.eval().state_dict(), file_name)

### Carregando os pesos

In [52]:
file_name = 'pesos_phil9-v3_U_1024-512.txt'

In [63]:
my_data_test = genfromtxt('../SPIHT_dataset/david/SPIHT_bits_with_context_david_Y80_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))

yc = bitstream_test > 0 # encode the second part
Xc = context_test >= 0 # truncated X for coding
Xc = Xc.astype(int)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)
N = Xc[0, :].size

In [54]:
model = MLP_N_1024_512_1(N)
model.load_state_dict(torch.load(file_name))

<All keys matched successfully>

In [40]:
Xc_tensor = torch.from_numpy(Xc[0:, :])
yc_tensor = torch.from_numpy(yc[0:])
predictions = model(Xc_tensor.float())
criterion=Log2BCELoss()
tamanho_medio = criterion(predictions,yc_tensor.float())
print(f"""comprimento médio de código final no modelo treinado: {tamanho_medio}""")

comprimento médio de código final no modelo treinado: 0.9485439658164978


In [55]:
bit_list = [591889,301037,131569,51154]
for i in bit_list:
    Xc_tensor = torch.from_numpy(Xc[0:i, :])
    yc_tensor = torch.from_numpy(yc[0:i])
    predictions = model(Xc_tensor.float())
    criterion=Log2BCELoss()
    tamanho_medio = criterion(predictions,yc_tensor.float())
    print(f"""comprimento médio de código final no modelo treinado: {tamanho_medio}""")

comprimento médio de código final no modelo treinado: 0.9572733044624329
comprimento médio de código final no modelo treinado: 0.961675226688385
comprimento médio de código final no modelo treinado: 0.9664247632026672
comprimento médio de código final no modelo treinado: 0.9623333811759949


In [64]:
perfect_AC(yc[0:1042536],context_coding(Xc[0:1042536, :],context_training(Xc[0:, :],yc[0:])))

0.9040812081822108

### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [ ]:
for param in net.parameters():
    print(param.data)

In [17]:
C.T, C0

NameError: name 'C' is not defined